In [1]:
#!/usr/bin/env python
import tensorflow as tf
import numpy as np
import h5py
import time
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from skimage.measure import compare_ssim
from google.colab import drive    # in ordre to mount drive to colab
drive.mount('/content/gdrive')

gpu_options = tf.GPUOptions(allow_growth=True)
session_config = tf.ConfigProto(log_device_placement=False, gpu_options=gpu_options, allow_soft_placement=True)
sess = tf.Session(config=session_config)

patch_width = 64

batch_size = 30
learning_rate = 1.0e-4
width = 512
beta1 = 0.9
beta2 = 0.999

num_epoch = 10


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
def normalize(batch, lower = -160.0, upper = 240.0):
    #batch = 800 * batch - 400
    #batch = (batch - lower) / (upper - lower)
    batch[batch > 1.0] = 1.0
    batch[batch < 0.0] = 0.0
    [X, Y] = np.mgrid[0:width, 0:width]
    xpr = X - int(width) // 2
    ypr = Y - int(width) // 2
    radius = width // 2
    reconstruction_circle = (xpr ** 2 + ypr ** 2) <= radius ** 2
    batch[~reconstruction_circle] = 0.
    #batch = (batch - np.min(batch)) / (np.max(batch) - np.min(batch))
    return np.squeeze(batch)

In [0]:
def read_data(file_name):
    f = h5py.File(file_name, 'r')
    data = np.array(f['data'])[0:10000]
    label = np.array(f['label'])[0:10000]
    f.close()
    return data, label


In [0]:
def network(inputs, padding='same'):
    outputs1 = tf.layers.conv2d(inputs, 32, 3, padding=padding, kernel_initializer=tf.contrib.layers.xavier_initializer(), name='conv1', use_bias=False)
    outputs2 = tf.nn.relu(outputs1)
    

    outputs2 = tf.layers.conv2d(outputs2, 32, 3, padding=padding, kernel_initializer=tf.contrib.layers.xavier_initializer(), name='conv2', use_bias=False)
    outputs3 = tf.nn.relu(outputs2)

    outputs3 = tf.layers.conv2d(outputs3, 32, 3, padding=padding, kernel_initializer=tf.contrib.layers.xavier_initializer(), name='conv3', use_bias=False)
    outputs4 = tf.nn.relu(outputs3)

    outputs4 = tf.layers.conv2d(outputs4, 32, 3, padding=padding, kernel_initializer=tf.contrib.layers.xavier_initializer(), name='conv4', use_bias=False)
    outputs5 = tf.nn.relu(outputs4)
    
    outputs5 = tf.layers.conv2d_transpose(outputs5, 32, 3, padding=padding, kernel_initializer=tf.contrib.layers.xavier_initializer(), name='deconv5', 
                                          use_bias=False)
    outputs5 = tf.nn.relu(outputs5)
    outputs5 = tf.concat([outputs3, outputs5], -1)

    outputs6 = tf.layers.conv2d_transpose(outputs5, 32, 3, padding=padding, kernel_initializer=tf.contrib.layers.xavier_initializer(), name='deconv6', use_bias=False)
    outputs6 = tf.nn.relu(outputs6)    
    outputs6 = tf.concat([outputs2, outputs6], -1)

    outputs7 = tf.layers.conv2d_transpose(outputs6, 32, 3, padding=padding, kernel_initializer=tf.contrib.layers.xavier_initializer(), name='deconv7', use_bias=False)
    outputs7 = tf.nn.relu(outputs7)
    outputs7 = tf.concat([outputs1, outputs7], -1)

    outputs8 = tf.layers.conv2d_transpose(outputs7, 1, 3, padding=padding, kernel_initializer=tf.contrib.layers.xavier_initializer(), name='deconv8', use_bias=False)
    outputs = tf.nn.relu(outputs8)
    
    return outputs 

In [5]:
X = tf.placeholder(dtype=tf.float32, shape=[batch_size, patch_width, patch_width, 1])
real_data = tf.placeholder(dtype=tf.float32, shape=[batch_size, patch_width, patch_width, 1])
lr = tf.placeholder(tf.float32, shape=[])
test_data_placeholder = tf.placeholder(dtype=tf.float32, shape=[1, width, width, 1])
with tf.variable_scope('Model', reuse=tf.AUTO_REUSE) as scope_generator_model:
    Y_ = network(X)
    Y_test = network(test_data_placeholder)
    mse = tf.reduce_mean(tf.squared_difference(Y_, real_data))
    gen_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Model')
    gen_train_op = tf.train.AdamOptimizer(learning_rate=lr).minimize(mse, var_list=gen_variables)
Patient_IDs = 'L109'
Test_Patient_IDs = ['L143']
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())
#saver.restore(sess, '/content/gdrive/My Drive/hands_on3/model/SIGmathmodel100.ckpt')
filename = "/content/gdrive/My Drive/hands_on3/" + str(Patient_IDs) + "_patch.h5"
data, label = read_data(filename)
for iteration in range(num_epoch):
    val_lr = learning_rate / np.sqrt(iteration + 1)
    for ID in Patient_IDs:

        data, label = shuffle(data, label)
        num_batches = data.shape[0] // batch_size
        print("###################################")
        print("epoch:{}".format(iteration))
        print("learning rate:{}".format(val_lr))
        print("num batch:{}".format(num_batches))
        print("###################################")
        for i in range(num_batches):
            batch_data = np.expand_dims(np.squeeze(data[i * batch_size: (i + 1) * batch_size]), -1)
            batch_label = np.expand_dims(np.squeeze(label[i * batch_size: (i + 1) * batch_size]), -1)
            _mse, _ = sess.run([mse, gen_train_op], feed_dict={real_data: (batch_label),
                                                                             X: (batch_data), lr: val_lr})

            print('Epoch: %d - %d - mse %.6f: ' % (iteration, i, _mse))
        break
    saver.save(sess, '/content/gdrive/My Drive/hands_on3/model/SIGmathmodel100.ckpt')
    print("testing dataset")
    for ID in Test_Patient_IDs:
        print("Testing Patient:{}".format(ID))
        filename = "/content/gdrive/My Drive/hands_on3/" + str(ID) + ".h5"
        c = 0
        f_t = h5py.File(filename, 'r')
        data_t, label_real = read_data(filename)
        f_t.close()
        for c in range(100,110):
            print(c)
            test_img = np.squeeze(label_real[c * 1: (c + 1) * 1])
            test_img = np.expand_dims(test_img, 0)
            test_img = np.expand_dims(test_img, -1)
            test_fbp = np.squeeze(data_t[c * 1: (c + 1) * 1])
            test_fbp = np.expand_dims(test_fbp, 0)
            test_fbp = np.expand_dims(test_fbp, -1)
            with tf.variable_scope('generator_model') as scope:
                scope.reuse_variables()
                estimated = sess.run(Y_test, feed_dict={test_data_placeholder: test_fbp})
                t_r = normalize(np.squeeze(test_img))
                e_1 = normalize(np.squeeze(estimated))
                f_n = '/content/gdrive/My Drive/hands_on3/denoise/' + str(c) + str(i) + 'validation.jpg'
                plt.imsave(f_n, e_1, cmap='gray')
                f_n = '/content/gdrive/My Drive/hands_on3/denoise/' + str(c) + str(i) + 'real_.jpg'
                plt.imsave(f_n, t_r, cmap='gray')

                z = np.mean((t_r - e_1) ** 2)
                print(z)




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Use `tf.keras.layers.Conv2DTranspose` instead.
###################################
epoch:0
learning rate:0.0001
num batch:333
###################################
Epoch: 0 - 0 - mse 0.083957: 
Epoch: 0 - 1 - mse 0.137157: 
Epoch: 0 - 2 - mse 0.082065: 
Epoch: 0 - 3 - mse 0.081263: 
Epoch: 0 - 4 - mse 0.097657: 
Epoch: 0 - 5 - mse 0.076888: 
Epoch: 0 - 6 - mse 0.083366: 
Epoch: 0 - 7 - mse 0.052900: 
Epoch: 0 - 8 - mse 0.049640: 
Epoch: 0 - 9 - mse 0.051798: 
Epoch: 0 - 10 - mse 0.044431: 
Epoch: 0 - 11 - mse 0.044863: 
Epoch: 0 - 12 - mse 